## NI Claims Text Cleaning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import s3fs
import spacy

In [2]:
import nltk
# from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
# nltk.download() #<<< one time effort

### Load Data

In [3]:
bucket = 's3://cortex-ni-wc-claims'
file = 'WC_DeEscl_Data_Build_wClstr_v2.csv'
# col_type = {'trip_id': int, 'avg_speed': float, 'max_speed': float, 'avg_speed_limit': float, 
#             'sample_count':int, 'driver_name':object}
# df = pd.read_csv(f'{bucket}/{file}', dtype = col_type)

df = pd.read_csv(f'{bucket}/{file}')
print('Data shape: ', df.shape)

/mnt/miniconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (86,87,88,89) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Data shape:  (544529, 101)


In [4]:
df.head(10)

Year  Claim_ID  Claim_Num Claim_System_Entry_Dte  Report_dte  Injury_Dte  \
0  2013   2518406  413A69752             2013-01-02  2013-01-01  2012-12-31   
1  2013   2539906  390A93167             2013-01-11  2013-01-10  2013-01-09   
2  2013   2551806  949A85874             2013-01-17  2013-01-16  2013-01-14   
3  2013   2555306  14CA14539             2013-01-29  2013-01-28  2013-01-27   
4  2013   2555906  413A71888             2013-01-31  2013-01-29  2012-10-07   
5  2013   2557406  205A40639             2013-02-05  2013-02-03  2013-02-02   
6  2013   2558006  608A58238             2013-02-06  2013-02-05  2013-01-23   
7  2013   2558106  413A72486             2013-02-06  2013-02-05  2012-03-01   
8  2013   2558506  80DA84769             2013-02-07  2013-02-06  2013-01-29   
9  2013   2563306  608A59396             2013-02-22  2013-02-21  2013-02-20   

  Claim_Status_Cde Current_PMT_Cde Current_Controverted_I  \
0                F              07                    NaN   
1                F              01                    NaN   
2                F              0P                      Y   
3                F              0P                    NaN   
4                F              04                    NaN   
5                F              01                    NaN   
6                F              M1                    NaN   
7                F              04                    NaN   
8                F              05                    NaN   
9                F              M7                      Y   

   current_total_incurred  ...  PrimNOI_km_6c  PrimNOI_km_12c  PrimNOI_pam_6c  \
0                     0.0  ...              5               8               4   
1                  5747.0  ...              5               8               4   
2                     0.0  ...              5               8               4   
3                     0.0  ...              1               4               6   
4                   495.0  ...              1               8               4   
5                     0.0  ...              1               4               6   
6                   397.0  ...              1               4               6   
7                   258.0  ...              1              10               6   
8                     0.0  ...              5               8               4   
9                   756.0  ...              1               4               6   

  PrimNOI_pam_7c PrimNOI_pam_9c  Occur_km_3c Occur_km_4c Occur_km_6c  \
0              5              5            1           1           5   
1              5              5            1           1           5   
2              5              5            1           1           4   
3              7              9            1           1           5   
4              5              8            1           1           4   
5              7              9            1           1           5   
6              7              9            1           1           4   
7              7              8            3           3           1   
8              5              5            3           3           1   
9              7              9            3           3           1   

   Occur_pam_4c Occur_pam_9c  
0             4            6  
1             4            6  
2             4            9  
3             4            6  
4             4            9  
5             4            6  
6             4            9  
7             2            2  
8             2            2  
9             2            5  

[10 rows x 101 columns]

In [5]:
# fill missing in the notes
accident_notes = df['Accident_Note_Text']
accident_notes= accident_notes.fillna('unknown')
injury_notes = df['Injury_Note_Text']
injury_notes = injury_notes.fillna('unknown')

In [6]:
accident_notes.head(10)

0    ASSOICATE HAS BEEN INJURING HERSELF ON AND OFF...
1    WHEN WORKING IN THE LIGHTBULB AISLE ROCHELLE N...
2                             CAUSEUNSPECIFIED MEDICAL
3    WALKED UP ON THE ASSOCIATE SAYING SHE COULDNT ...
4    JOSH WAS TAKING AN ITEM OUT OF THE MICROWAVE A...
5    DROPPED HOME DEPOT KNIFE IN BREAK ROOM BENT DO...
6                                              unknown
7    EMPLOYEE STATES THAT HE FEELS PAIN IN BOTH WRISTS
8                              JERKED LEFT ARM TO HARD
9    ASSOCIATE FELT LIGHT HEADED AND FAINT FROM LIF...
Name: Accident_Note_Text, dtype: object

In [7]:
injury_notes.head(10)

0    unknown
1    unknown
2    unknown
3    unknown
4    unknown
5    unknown
6    unknown
7    unknown
8    unknown
9    unknown
Name: Injury_Note_Text, dtype: object

### Text Cleaning

In [8]:
from nltk.corpus import stopwords
from ast import literal_eval
import re
# from autocorrect import spell

In [44]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = stopwords.words('english')
keep = {'not'}
STOPWORDS = [e for e in STOPWORDS if e not in keep]
extra_stopwords = ['left', 'right', 'employee', 'emp', 'ee', 'associate', 'iw']
STOPWORDS = set(STOPWORDS + extra_stopwords)
# !!!may need to revise the stop words list later

def text_prepare(text):
    """
        input: a string
        
        return: cleaned string
    """
    text = text.lower() # lowercase text
    text = re.sub("n't", " not", text)
    text = re.sub(REPLACE_BY_SPACE_RE, " ", text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(BAD_SYMBOLS_RE, "", text) # delete symbols which are in BAD_SYMBOLS_RE from text
    word_tokens = word_tokenize(text) 
    word_tokens = [w for w in word_tokens if not w in STOPWORDS] # delete stopwords from text
  
    stemmer = nltk.stem.WordNetLemmatizer()
    for i in range(len(word_tokens)):
        # stemming + spelling correction
        
#          word_tokens[i] = token.lemma_( word_tokens[i])
        word_tokens[i] = stemmer.lemmatize(stemmer.lemmatize(word_tokens[i]), 'v') #lemmatize both noun and verb
#         word_tokens[i] = stemmer.lemmatize(spell(word_tokens[i]))
        
    text = " ".join(word_tokens)
    return text

In [45]:
import time
start_time = time.time()
accident_cleaned= accident_notes.apply(text_prepare)
print("--- %s seconds ---" % (time.time() - start_time))

--- 128.57029128074646 seconds ---


In [46]:
start_time = time.time()
injury_cleaned = injury_notes.apply(text_prepare)
print("--- %s seconds ---" % (time.time() - start_time))

--- 78.04394030570984 seconds ---


In [48]:
accident_cleaned[:10]

0       assoicate injure restriction say keyboard drop
1    work lightbulb aisle rochelle notice ache thum...
2                             causeunspecified medical
3    walk say couldnt see walk eye station wash eye th
4                    josh take item microwave fell arm
5    drop home depot knife break room bend pick kni...
6                                              unknown
7                                state feel pain wrist
8                                        jerk arm hard
9                           felt light head faint lift
Name: Accident_Note_Text, dtype: object

In [49]:
accident_cleaned.to_frame().describe()

Accident_Note_Text
count              544529
unique             519357
top               unknown
freq                 4689

In [50]:
injury_cleaned[:10]

0    unknown
1    unknown
2    unknown
3    unknown
4    unknown
5    unknown
6    unknown
7    unknown
8    unknown
9    unknown
Name: Injury_Note_Text, dtype: object

In [19]:
injury_cleaned.to_frame().describe()

Injury_Note_Text
count            544529
unique           191919
top             unknown
freq             138154

In [54]:
cleantxt_df = pd.DataFrame({'accident_txt_cleaned': accident_cleaned, 
                       'injury_txt_cleaned': injury_cleaned, 
                       'deEscInd3000': df['deEscInd3000']}) 
cleantxt_df.head()

accident_txt_cleaned injury_txt_cleaned  \
0     assoicate injure restriction say keyboard drop            unknown   
1  work lightbulb aisle rochelle notice ache thum...            unknown   
2                           causeunspecified medical            unknown   
3  walk say couldnt see walk eye station wash eye th            unknown   
4                  josh take item microwave fell arm            unknown   

   deEscInd3000  
0             1  
1             0  
2             1  
3             1  
4             1

In [58]:
# save cleaned text with response
filename = 'cleantxt_df.csv'
cleantxt_df.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)

In [55]:
df['Acc_Txt_Clned'] = accident_cleaned
df['Inj_Txt_Clned'] =  injury_cleaned

In [57]:
df.shape

(544529, 103)

In [59]:
# save cleaned text with the original data fields
filename = 'WC_DeEscl_Data_Build_wClstr_v2_txtcln.csv'
df.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)